In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
# import http.client
# import pycurl 
import time
from scipy.stats import linregress
from datetime import datetime

# Import API key
from config import ATTOM_API

In [29]:
# Read in historical mortgage rate
mortgage_data = pd.read_csv('Resources/morgage_rates.csv')
year_mortgage = []
month_mortgage = []
mortgage_rate = []

# Loop to limit data for 2019 & 2020
for i,j in mortgage_data.iterrows():
    if j['DATE']>='2019%' and j['DATE']<'2021%':
        year_mortgage.append(pd.to_datetime(j['DATE']).year)
        month_mortgage.append(pd.to_datetime(j['DATE']).month)
        mortgage_rate.append(j['MORTGAGE30US'])


# Create mortgage rate DF grouped by year and month to get the mean
mortgage_rates = pd.DataFrame({"year":year_mortgage, "month":month_mortgage, 
                        "mortgage_rate":mortgage_rate}).groupby(['year', 'month'])

# Export csv of calculated monthly rates
mortgage_rates.mean().to_csv('Resources/monthly_morgage_rates.csv')

In [2]:
# Establish API URL
property_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/"

sales_trend = "salestrend/snapshot?geoid=ZI"

zipcode = "32608"

sales_trend_dates = "&interval=monthly&startyear=2019&endyear=2020&startmonth=january&endmonth=december"

headers = {'Accept': 'application/json', 'apikey': ATTOM_API}

# Make API call
sales_response = requests.post( f'{property_url}{sales_trend}{zipcode}{sales_trend_dates}', 
                                headers = headers)
# Establish variable of json and Display response for reference
print(sales_response.status_code)
sales = sales_response.json()
sales

200


{'status': {'version': '1.0.0',
  'code': 0,
  'msg': 'SuccessWithResult',
  'total': 24},
 'salestrends': [{'location': {'geoID': 'ZI32608'},
   'daterange': {'start': 'January 2019',
    'end': 'January 2019',
    'interval': 'monthly'},
   'SalesTrend': {'homesalecount': 50,
    'avgsaleprice': 353490.0,
    'medsaleprice': 153550.0},
   'vintage': {'pubDate': '2021-4-26'}},
  {'location': {'geoID': 'ZI32608'},
   'daterange': {'start': 'March 2019',
    'end': 'March 2019',
    'interval': 'monthly'},
   'SalesTrend': {'homesalecount': 42,
    'avgsaleprice': 224033.0,
    'medsaleprice': 163500.0},
   'vintage': {'pubDate': '2021-4-26'}},
  {'location': {'geoID': 'ZI32608'},
   'daterange': {'start': 'April 2019',
    'end': 'April 2019',
    'interval': 'monthly'},
   'SalesTrend': {'homesalecount': 95,
    'avgsaleprice': 218566.0,
    'medsaleprice': 184300.0},
   'vintage': {'pubDate': '2021-4-26'}},
  {'location': {'geoID': 'ZI32608'},
   'daterange': {'start': 'May 2019',
  

In [35]:
# Establish variable for information desired
year = []
month = []
zip_code_sales = []
total_sales = []
average_price = []
median_price = []

# Loop through API data and append to variables
for i in sales['salestrends']:
    year.append(pd.to_datetime(i['daterange']['start']).year)
    month.append(pd.to_datetime(i['daterange']['start']).month)
    zip_code_sales.append(i['location']['geoID'][2:7])
    total_sales.append(i['SalesTrend']['homesalecount'])
    average_price.append(i['SalesTrend']['avgsaleprice'])
    median_price.append(i['SalesTrend']['medsaleprice'])


# Create new DF with data
sales_data = pd.DataFrame({"zipcode": zip_code_sales, "year":year, "month":month, 
                        "total_sales":total_sales, "avg_sale_price":average_price,
                        "med_sale_price":median_price})

# Read in monthly mortgage rates
monthly_rates = pd.read_csv('Resources/monthly_morgage_rates.csv')

# Add mortgage rates to sales data DF
sales_data['mortgage_rate'] = monthly_rates['mortgage_rate']

sales_data

,zipcode,year,month,total_sales,avg_sale_price,med_sale_price,mortgage_rate
0,32608,2019,1,50,353490.0,153550.0,4.4640
1,32608,2019,3,42,224033.0,163500.0,4.3700
2,32608,2019,4,95,218566.0,184300.0,4.2650
3,32608,2019,5,102,221271.0,174950.0,4.1425
4,32608,2019,6,91,229545.0,171000.0,4.0720
5,32608,2019,7,105,208898.0,163800.0,3.8025
6,32608,2019,8,78,205419.0,168250.0,3.7650
7,32608,2019,9,45,179327.0,137000.0,3.6160
8,32608,2019,10,53,238298.0,225000.0,3.6050
9,32608,2019,11,54,287678.0,260500.0,3.6880


In [124]:
community_url = "https://api.gateway.attomdata.com/communityapi/v2.0.0"

area = "/area/full?AreaId=ZI"

zipcode = "32608"

headers = {'Accept': 'application/json', 'apikey': ATTOM_API}

area_response = requests.post( f'{community_url}{area}{zipcode}', headers = headers)

print(area_response.status_code)
area = area_response.json()
area

200


{'response': {'inputparameter': {'AreaId': 'ZI32608',
   'package': 'full',
   'resource': 'area',
   'service': 'Community'},
  'result': {'package': {'descr': '',
    'item': [{'age00_04': '3791',
      'age05_09': '3125',
      'age10_14': '2490',
      'age15_19': '4922',
      'age20_24': '7919',
      'age25_29': '1837',
      'age30_34': '5471',
      'age35_39': '4557',
      'age40_44': '2853',
      'age45_49': '2183',
      'age50_54': '1978',
      'age55_59': '2095',
      'age60_64': '2040',
      'age65_69': '1995',
      'age70_74': '1550',
      'age75_79': '1050',
      'age80_84': '677',
      'ageavepy_10': '36.00',
      'ageavepy_5': '35.00',
      'agegt85': '674',
      'aimcy25_44': '52741',
      'aimcy45_64': '71554',
      'aimcygt_65': '58192',
      'aimcylt_25': '18757',
      'airport': 'Gainesville Regional',
      'airportdist': '9',
      'airx': '129',
      'ancestamer': '1503',
      'ancestamind': '101',
      'ancestasian': '5607',
      'anceste

In [145]:
print(area['response']['result']['package']['item'][0]['avg_prop_tax'])

3629


In [152]:
print(area['response']['inputparameter']['AreaId'][2:7])

32608


In [154]:
zip_code_area = area['response']['inputparameter']['AreaId'][2:7]
prop_tax = area['response']['result']['package']['item'][0]['avg_prop_tax']




area_data = pd.DataFrame({"zipcode": zip_code_area, "avg_prop_tax":prop_tax})
area_data

ValueError: If using all scalar values, you must pass an index